# Inconsistency Table (AFAD)

In [1]:
%load_ext watermark
%watermark -p torch,pandas,numpy

torch 1.5.1
pandas 1.0.1
numpy 1.18.1


In [2]:
import torch
import numpy as np
import pandas as pd
import os

In [3]:
def count_inconsistency(ary):
    cnt = 0
    last = ary[0]
    for i in ary[1:]:
        if i > last + 1e-05:
            cnt += 1
        last = i
    return cnt


def read_labels_from_log(path):
    with open(path, 'r') as f:
        cont = f.read()
    return np.array(cont.split(',')).astype(int)

In [4]:
basepath = '../experiment-logs/afad'
test_path = '../datasets/afad_test.csv'

In [5]:
test_labels = pd.read_csv(test_path, usecols=['age']).values.reshape(-1)
test_labels[:15]

array([24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24])

In [6]:
# Coral

for i in range(3):
    
    ordinal_path = os.path.join(basepath, f'afad-coral__seed{i}')
    
    test_proba_path_seed00 = os.path.join(ordinal_path, 'test_allprobas.tensor')
    t = torch.load(test_proba_path_seed00)

    inconsistency_counts = [count_inconsistency(t[i]) for i in range(t.size()[0])]
    avg = np.mean(inconsistency_counts)
    print(f'Average Inconsistency Seed {i}: {avg:.2f}')
    

Average Inconsistency Seed 0: 0.00
Average Inconsistency Seed 1: 0.00
Average Inconsistency Seed 2: 0.00


In [7]:
# Ordinal

for i in range(3):
    
    ordinal_path = os.path.join(basepath, f'afad-ordinal__seed{i}/')
    
    test_proba_path_seed00 = os.path.join(ordinal_path, 'test_allprobas.tensor')
    t = torch.load(test_proba_path_seed00)

    inconsistency_counts = [count_inconsistency(t[i]) for i in range(t.size()[0])]
    avg = np.mean(inconsistency_counts)
    print(f'Average Inconsistency Seed {i}: {avg:.2f}')
    

Average Inconsistency Seed 0: 1.97
Average Inconsistency Seed 1: 1.91
Average Inconsistency Seed 2: 1.17


In [8]:
for i in range(0, 3):

    when_correct_cnt = []
    when_incorrect_cnt = []
    
    ordinal_path = os.path.join(basepath, f'afad-ordinal__seed{i}/')

    test_proba_path_seed = os.path.join(ordinal_path, 'test_allprobas.tensor')
    t = torch.load(test_proba_path_seed)
    
    test_proba_path = os.path.join(ordinal_path, 'test_predictions.log')
    lab = read_labels_from_log(test_proba_path)    
    
    for j in range(test_labels.shape[0]):
        if test_labels[j] != lab[j]:
            when_incorrect_cnt.append(count_inconsistency(t[j]))
        else:
            when_correct_cnt.append(count_inconsistency(t[j]))
            
    avg = np.mean(when_correct_cnt)
    print(f'Seed 0{i} when correct: {avg:.2f}')
    
    avg = np.mean(when_incorrect_cnt)
    print(f'Seed 0{i} when incorrect: {avg:.2f}')

Seed 00 when correct: 1.88
Seed 00 when incorrect: 1.98
Seed 01 when correct: 1.81
Seed 01 when incorrect: 1.92
Seed 02 when correct: 1.02
Seed 02 when incorrect: 1.19
